### Config file generator
<img src='../graphics/loop_sm.png'>
This simple notebook provides a tool for generating map2loop config files, one of the more annoying parts of the process.
   
1. It first gets the project name and destination projection system (as EPSG number)
2. then asks for the paths to the different shape files (for WFS you will have to edit this manually afterwards)   
3. Finally for each input file, you have to define which fields to use, and which keyords to look for in some of these fields   
   

Finally, make a copy of the <a href="0.%20Generic%20config%20file.ipynb" target="_blank">0. Generic config file notebook</a> and update the **test_data_name** parameter
   
Lots of the parameters not related to the GIS layers are hard-wired by this notebook, but can be easily overridden in the notebooks and are much less fiddly.

In [ ]:
from ipyfilechooser import FileChooser
import geopandas as gpd
import ipywidgets as widgets
import pandas as pd
import os
from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))
    
pd.options.display.max_columns = None
style = {'description_width': 'initial'}
c_l= {
#Orientations
  "d": "dip",                  #field that contains dip information
  "dd": "azimuth",             #field that contains dip direction information
  "sf": 'codedescpt',             #field that contains information on type of structure
  "bedding": 'bedding',            #text to search for in field defined by sf code to show that this is a bedding measurement
  "otype": 'dip direction',            #flag to determine measurement convention (currently 'strike' or 'dip direction')
  "bo": "deform",             #field that contains type of foliation
  "btype": 'BEOI',            #text to search for in field defined by bo code to show that this is an overturned bedding measurement
#stratigraphy
  "g": 'sub_provin',               #field that contains coarser stratigraphic coding
  "g2": 'province',              #field that contains alternate coarser stratigraphic coding if 'g' is blank
  "c": 'unit_name',                 #field that contains finer stratigraphic coding
  "ds": 'deposition',           #field that contains information about lithology
  "u": 'nsw_code',             #field that contains alternate stratigraphic coding (not used??)
  "r1": 'class',           #field that contains  extra lithology information
  "r2": 'igneous_ty',           #field that contains even more lithology information
  "sill": 'sill',              #text to search for in field defined by ds code to show that this is a sill
  "intrusive": 'intrusive',    #text to search for in field defined by r1 code to show that this is an intrusion
  "volcanic": 'volcanic',      #text to search for in field defined by ds code to show that this is an volcanic (not intrusion)
#mineral deposits
  "msc": 'occurrence',          #field that contains site code of deposit
  "msn": 'deposit_na',         #field that contains short name of deposit
  "mst": 'comm_type',         #field that contains site type of deposit
  "mtc": 'major_comm',         #field that contains target commodity of deposit
  "mscm": 'ore_min',        #field that contains site commodity of deposit
  "mcom": 'comm_type',        #field that contains commodity group of deposit
  "minf": 'CONSTR',    #text to search for in field defined by mst code that shows site to ignore
#timing
  "min": 'top_end_ag',         #field that contains minimum age of unit defined by ccode
  "max": 'base_start',         #field that contains maximum age of unit defined by ccode
#faults and folds
  "f": 'boundaryty',              #field that contains information on type of structure
  "fault": 'Fault',            #text to search for in field defined by f code to show that this is a fault
  "ff": 'codedescpt',  # field that contains information on type of structure
  "fold": 'cline',  #text to search for in field defined by ff code to show that this is a fold axial trace
  "fdip": 'dip',               # field for numeric fault dip value
  "fdipnull": '0',         # text to search for in field defined by fdip to show that this has no known dip
  "fdipdir": 'faultdipdi',        # field for text fault dip direction value 
  "fdipdir_flag": 'alpha',        # flag for text fault dip direction type num e.g. 045 or alpha e.g. southeast    
  "fdipest": 'faultdipan',        # field for text fault dip estimate value
  "fdipest_vals": 'Moderate,Listric,Steep,Vertical',        # text to search for in field defined by fdipest to give fault dip estimate in increasing steepness
  "n": 'name',                 #field that contains information on name of fault (not used??)
  "t": 'codedescpt',                 #field that contains information on type of fold
  "syn": 'Syncline',           #text to search for in field defined by t to show that this is a syncline
#ids
  "o": 'unique_id',             #field that contains unique id of geometry object
  "gi": 'objectid'            #field that contains unique id of structure point
}

In [ ]:
test_data_name_choice=widgets.Text(
    placeholder='Test',
    value='Test',
    description='Project Name:',
    disabled=False,
    style=style,
    tooltip='Name of directory to hold config file (relative path)'
)
display(test_data_name_choice)

epsg=pd.read_csv('../source_data/epsg.csv')

crs_choice=widgets.Dropdown(
    options=epsg['epsg'],
    description='CRS EPSG:',
    disabled=False
)
display(crs_choice)


In [ ]:
test_data_name=test_data_name_choice.value
epsg_code=crs_choice.value.split(":")
dst_crs='epsg:'+epsg_code[0]
printmd('**'+test_data_name+", "+dst_crs+'**')

In [ ]:
minx_choice=widgets.Text(
    placeholder='0',
    value='0',
    description='min x:',
    disabled=False,
    style=style
)
display(minx_choice)

maxx_choice=widgets.Text(
    placeholder='1',
    value='1',
    description='max x:',
    disabled=False,
    style=style
)
display(maxx_choice)

miny_choice=widgets.Text(
    placeholder='0',
    value='0',
    description='min y:',
    disabled=False,
    style=style
)
display(miny_choice)

maxy_choice=widgets.Text(
    placeholder='1',
    value='1',
    description='max y:',
    disabled=False,
    style=style
)
display(maxy_choice)


In [ ]:
minx=minx_choice.value
maxx=maxx_choice.value
miny=miny_choice.value
maxy=maxy_choice.value

structure_file_m = FileChooser('../')
structure_file_m.title = '<b>Choose Bedding Orientation point file</b>'
display(structure_file_m)

fault_file_m = FileChooser('../')
fault_file_m.title = '<b>Choose Fault polyline file</b>'
display(fault_file_m)

fold_file_m = FileChooser('../')
fold_file_m.title = '<b>Choose Fold polyline file</b>'
display(fold_file_m)

geology_file_m = FileChooser('../')
geology_file_m.title = '<b>Choose Geology Polygon file</b>'
display(geology_file_m)

mindep_file_m = FileChooser('../')
mindep_file_m.title = '<b>Choose Mineral Deposit file</b>'
display(mindep_file_m)



In [ ]:
if(str(structure_file_m.selected_path) =='None'):
    structure_file=''
else:
    structure_file=structure_file_m.selected_path.replace('\\','/')+'/'+structure_file_m.selected_filename
if(str(fault_file_m.selected_path) =='None'):
    fault_file=''
else:
    fault_file=fault_file_m.selected_path.replace('\\','/')+'/'+fault_file_m.selected_filename
if(str(fold_file_m.selected_path) =='None'):
    fold_file=''
else:
    fold_file=fold_file_m.selected_path.replace('\\','/')+'/'+fold_file_m.selected_filename
if(str(geology_file_m.selected_path) =='None'):
    geology_file=''
else:
    geology_file=geology_file_m.selected_path.replace('\\','/')+'/'+geology_file_m.selected_filename
if(str(mindep_file_m.selected_path) =='None'):
    mindep_file=''
else:
    mindep_file=mindep_file_m.selected_path.replace('\\','/')+'/'+mindep_file_m.selected_filename

display(structure_file,fault_file,fold_file,geology_file,mindep_file)

In [ ]:
if(not structure_file==''):
    structure=gpd.read_file(structure_file)
    display(structure.head())

printmd('**Based on file: '+structure_file+'**')
printmd('\n**Field that contas unique id of structure point**')
gi_choice=widgets.Dropdown(
    options=structure.columns,
    description='Unique ID',
    disabled=False

)
display(gi_choice)

printmd('**Field that contains information on type of structure**')

sf_choice=widgets.Dropdown(
    options=structure.columns,
    description='Structure',
    disabled=False
)
display(sf_choice)

printmd('**Text to search for in field defined by type of structure code to show that this is a bedding measurement**')

bedding_choice=widgets.Text(
    placeholder='Bed',
    value='Bed',
    description='Bedding text',
    disabled=False
)
display(bedding_choice)

printmd('**Field that contains dip information**')

d_choice=widgets.Dropdown(
    options=structure.columns,
    description='Dip',
    disabled=False
)
display(d_choice)

printmd('**Field that contains dip direction information**')

dd_choice=widgets.Dropdown(
    options=structure.columns,
    description='Dip Direction',
    disabled=False
)
display(dd_choice)

printmd('**Flag to determine measurement convention**')

otype_choice=widgets.Dropdown(
    options=('dip direction','strike'),
    description='Convention',
    disabled=False
)
display(otype_choice)

printmd('**Field that contains polarity of foliation**')

bo_choice=widgets.Dropdown(
    options=structure.columns,
    description='Polarity',
    disabled=False
)
display(bo_choice)

printmd('**Text to search for in field defined by Polarity code to show that this is an overturned bedding measurement**')

btype_choice=widgets.Text(
    placeholder='Blah',
    value='Blah',
    description='Pol. text',
    disabled=False
)
display(btype_choice)



In [ ]:
c_l['gi']=gi_choice.value
c_l['sf']=sf_choice.value
c_l['bedding']=bedding_choice.value
c_l['d']=d_choice.value
c_l['dd']=dd_choice.value
c_l['otype']=otype_choice.value
c_l['bo']=bo_choice.value
c_l['btype']=btype_choice.value

if(not fault_file==''):
    fault=gpd.read_file(fault_file)
    display(fault.head())
printmd('**Based on file: '+fault_file+'**')

printmd('\n**Field that contains information on type of structure**')


f_choice=widgets.Dropdown(
    options=fault.columns,
    description='Fault field:',
    disabled=False
)
display(f_choice)

printmd('**Text to search for in field defined by Fault field code to show that this is a fault**')

fault_choice=widgets.Text(
    placeholder='Fault',
    value='Fault',
    description='Fault text:',
    disabled=False
)
display(fault_choice)

printmd('**Field for numeric fault dip value**')

fdip_choice=widgets.Dropdown(
    options=fault.columns,
    description='Dip:',
    disabled=False
)
display(fdip_choice)

printmd('**Text to search for in field defined by fdip to show that this has no known dip**')

fdipnull_choice=widgets.Text(
    placeholder='0',
    value='0',
    description='NULL dip:',
    disabled=False
)
display(fdipnull_choice)

printmd('**Field for text fault dip direction value**')

fdipdir_choice=widgets.Dropdown(
    options=fault.columns,
    description='Dip Direction:',
    disabled=False
)
display(fdipdir_choice)

printmd('**Flag for text fault dip direction type num e.g. 045 or alpha e.g. southeast**')

fdipdir_flag_choice=widgets.Dropdown(
    options=('alpha','num'),
    description='Dip Convention:',
    disabled=False,
    style=style
)
display(fdipdir_flag_choice)

printmd('**Field for text fault dip estimate value**')

fdipest_choice=widgets.Dropdown(
    options=fault.columns,
    description='Dip estimate:',
    disabled=False
)
display(fdipest_choice)

printmd('**Text to search for in field defined by fdipest to give fault dip estimate in increasing steepness**')

fdipest_vals_choice=widgets.Text(
    placeholder='Shallow, Steep',
    value='Shallow, Steep',
    description='Text Dips:',
    disabled=False
)
display(fdipest_vals_choice)

printmd('**Field that contains information on name of fault (not used??)**')

n_choice=widgets.Dropdown(
    options=fault.columns,
    description='Name:',
    disabled=False
)
display(n_choice)






In [ ]:
c_l['f']=f_choice.value
c_l['fault']=fault_choice.value
c_l['fdip']=fdip_choice.value
c_l['fdipnull']=fdipnull_choice.value
c_l['fdipdir']=fdipdir_choice.value
c_l['fdipdir_flag']=fdipdir_flag_choice.value
c_l['fdipest']=fdipest_choice.value
c_l['n']=n_choice.value

if(not fold_file==''):
    fold=gpd.read_file(fold_file)
    display(fold.head())
printmd('**Based on file:'+' '+fault_file+'**')

printmd('\n**Field that contains information on type of structure**')

ff_choice=widgets.Dropdown(
    options=fold.columns,
    description='Fold field:',
    disabled=False
)
display(ff_choice)

printmd('**Text to search for in field defined by ff code to show that this is a fold axial trace**')

fold_choice=widgets.Text(
    placeholder='Fold',
    value='Fold',
    description='Fold text:',
    disabled=False
)
display(fold_choice)

printmd('**Field that contains information on type of fold**')

t_choice=widgets.Dropdown(
    options=fold.columns,
    description='Polarity field:',
    disabled=False
)
display(t_choice)

printmd('**Text to search for in field defined by t to show that this is a syncline**')

syn_choice=widgets.Text(
    placeholder='syn',
    value='syn',
    description='Syncline text:',
    disabled=False
)
display(syn_choice)


In [ ]:
c_l['ff']=ff_choice.value
c_l['fold']=fold_choice.value
c_l['t']=t_choice.value
c_l['syn']=syn_choice.value

if(not geology_file==''):
    geology=gpd.read_file(geology_file)
    display(geology.head())
printmd('**Based on file:'+' '+geology_file+'**')

printmd('\n**Field that contains unique ID**')

o_choice=widgets.Dropdown(
    options=geology.columns,
    description='Unique ID',
    disabled=False
)
display(o_choice)

printmd('\n**Field that contains coarser stratigraphic coding**')

g_choice=widgets.Dropdown(
    options=geology.columns,
    description='Group',
    disabled=False
)
display(g_choice)

printmd('**Field that contains alternate coarser stratigraphic coding if Group is blank**')

g2_choice=widgets.Dropdown(
    options=geology.columns,
    description='Group alt',
    disabled=False
)
display(g2_choice)

printmd('**Field that contains finer stratigraphic coding**')

c_choice=widgets.Dropdown(
    options=geology.columns,
    description='Formation',
    disabled=False
)
display(c_choice)

printmd('**Field that contains information about lithology**')

ds_choice=widgets.Dropdown(
    options=geology.columns,
    description='Description:',
    disabled=False
)
display(ds_choice)

printmd('**Field that contains alternate stratigraphic coding (not used??)**')

u_choice=widgets.Dropdown(
    options=geology.columns,
    description='Strat alt',
    disabled=False
)
display(u_choice)

printmd('**Field that contains extra lithology information**')

r1_choice=widgets.Dropdown(
    options=geology.columns,
    description='Litho',
    disabled=False
)
display(r1_choice)

printmd('**Field that contains even more lithology information**')

r2_choice=widgets.Dropdown(
    options=geology.columns,
    description='Litho alt',
    disabled=False
)
display(r2_choice)

printmd('**Text to search for in field defined by ds code to show that this is a sill**')

sill_choice=widgets.Text(
    placeholder='sill',
    value='sill',
    description='Sill text',
    disabled=False
)
display(sill_choice)

printmd('**Text to search for in field defined by r1 code to show that this is an intrusion**')

intrusive_choice=widgets.Text(
    placeholder='intrusive',
    value='intrusive',
    description='Intrusive text',
    disabled=False
)
display(intrusive_choice)

printmd('**Text to search for in field defined by ds code to show that this is an volcanic (not intrusion)**')


volcanic_choice=widgets.Text(
    placeholder='volc',
    value='volc',
    description='Volc text',
    disabled=False
)
display(volcanic_choice)

printmd('**Field that contains minimum age of unit defined by Formation**')

min_choice=widgets.Dropdown(
    options=geology.columns,
    description='Min age',
    disabled=False
)
display(min_choice)

printmd('**Field that contains maximum age of unit defined by Formation**')

max_choice=widgets.Dropdown(
    options=geology.columns,
    description='Max age',
    disabled=False
)
display(max_choice)



In [ ]:
c_l['o']=o_choice.value
c_l['g']=g_choice.value
c_l['g2']=g2_choice.value
c_l['c']=c_choice.value
c_l['ds']=ds_choice.value
c_l['u']=u_choice.value
c_l['r1']=r1_choice.value
c_l['r2']=r2_choice.value
c_l['sill']=sill_choice.value
c_l['intrusive']=intrusive_choice.value
c_l['volcanic']=volcanic_choice.value
c_l['min']=min_choice.value
c_l['max']=max_choice.value

if(not mindep_file==''):
    mindep=gpd.read_file(mindep_file)
    display(mindep.head())
    
printmd('**Based on file:'+' '+mindep_file+'**')

printmd('\n**Field that contains site code of deposit**')

msc_choice=widgets.Dropdown(
    options=mindep.columns,
    description='Site code',
    disabled=False
)
display(msc_choice)

printmd('**Field that contains short name of deposit**')

msn_choice=widgets.Dropdown(
    options=mindep.columns,
    description='Name',
    disabled=False
)
display(msn_choice)

printmd('**Field that contains site type of deposit**')

mst_choice=widgets.Dropdown(
    options=mindep.columns,
    description='Site type',
    disabled=False
)
display(mst_choice)

printmd('**Field that contains target commodity of deposit**')

mtc_choice=widgets.Dropdown(
    options=mindep.columns,
    description='Major Comm',
    disabled=False
)
display(mtc_choice)

printmd('**Field that contains site commodity of deposit**')

mscm_choice=widgets.Dropdown(
    options=mindep.columns,
    description='Ore Min',
    disabled=False
)
display(mscm_choice)

printmd('**Field that contains commodity group of deposit**')

mcom_choice=widgets.Dropdown(
    options=mindep.columns,
    description='Commodity Group',
    disabled=False,
    style=style
)
display(mcom_choice)

printmd('**Text to search for in field defined by Site type code that shows site to ignore**')

minf_choice=widgets.Text(
    placeholder='Cons',
    value='Cons',
    description='Not mine text',
    disabled=False
)
display(minf_choice)


In [ ]:
c_l['msc']=msc_choice.value
c_l['msn']=msn_choice.value
c_l['mst']=mst_choice.value
c_l['mtc']=mtc_choice.value
c_l['mscm']=mscm_choice.value
c_l['mcom']=mcom_choice.value
c_l['minf']=minf_choice.value


if(not os.path.isdir('../'+test_data_name)):
    os.mkdir('../'+test_data_name)
f=open('../'+test_data_name+'/m2l_config.py',"w")

f.write("#ROI\n")
f.write("\n")
f.write("step_out=0.1   #padding around dtm to ensure reprojected dtm covers target area (in degrees)\n")
f.write("inset=0      #unused??\n")
f.write("\n")
f.write("#\n")
f.write("minx="+minx+"  #region of interest coordinates in metre-based system (or non-degree system)\n")
f.write("maxx="+maxx+"\n")
f.write("miny="+miny+"\n")
f.write("maxy="+maxy+"\n")
f.write("model_top=1200\n")
f.write("model_base=-3200\n")
f.write("\n")
f.write("#PATHS\n")
f.write("\n")
f.write("local_paths=False       #flag to use local or WFS source for data inputs (True = local)\n")
f.write("\n")
f.write("\n")
f.write("data_path=''\n")
f.write("clut_path=''\n")
f.write("base_url='https://gs-seamless.geoscience.nsw.gov.au/geoserver/ows?service=wfs&version=2.0.0&request=GetFeature&typeName='\n")
f.write("\n")
f.write("structure_file='"+structure_file+"'\n")
f.write("fault_file='"+fault_file+"'\n")
f.write("fold_file='"+fold_file+"'\n")
f.write("geology_file='"+geology_file+"'\n")
f.write("mindep_file='"+mindep_file+"'\n")
f.write("\n")
f.write("#CRS\n")
f.write("\n")
f.write("src_crs = 'epsg:4326'  # coordinate reference system for imported dtms (geodetic lat/long WGS84)\n")
f.write("dst_crs = '"+dst_crs+"' # coordinate system for data\n")
f.write("\n")
f.write("#CODES AND LABELS \n")
f.write("# these refer to specific fields (codes) in GIS layer or database that contain the info needed for these calcs and text substrings (labels) in the contents of these fields\n")
f.write("c_l=")
f.write(str(c_l)+"\n")
f.write("\n")
f.write("#DECIMATION\n")
f.write("\n")
f.write("orientation_decimate=0  #store every nth orientation (in object order) 0 = save all\n")
f.write("contact_decimate=10     #store every nth contact point (in object order) 0 = save all\n")
f.write("fault_decimate=5        #store every nth fault point (in object order) 0 = save all\n")
f.write("fold_decimate=5         #store every nth fold axial trace point (in object order) 0 = save all\n")
f.write("\n")
f.write("#INTERPOLATION\n")
f.write("\n")
f.write("gridx=50                #x grid dimensions (no of points, not distance) for interpolations\n")
f.write("gridy=50                #x grid dimensions (no of points, not distance) for interpolations\n")
f.write("scheme='scipy_rbf'      #interpolation scheme\n")
f.write("dist_buffer=5           #buffer distance for clipping points by faults (in metres or same units as dst_crs)\n")
f.write("intrusion_mode=0        # 1 all intrusions exluded from basal contacts, 0 only sills\n")
f.write("use_interpolations=False    # flag to use interpolated orientations or not.\n")
f.write("\n")
f.write("#ASSUMPTIONS\n")
f.write("\n")
f.write("pluton_dip=45           #surface dip of pluton contacts\n")
f.write("pluton_form='domes'     #saucers: \__+_+__/  batholith: +/       \+   domes: /  +  + \  pendant: +\_____/+\n")
f.write("fault_dip=90            #surface dip of faults\n")
f.write("\n")
f.write("#DERIVED AND FIXED PATHS\n")
f.write("\n")
f.write("m2m_cpp_path='../m2m_cpp/'\n")
f.write("\n")
f.write("graph_path=test_data_path+'graph/'\n")
f.write("tmp_path=test_data_path+'tmp/'\n")
f.write("#data_path=test_data_path+'data/'\n")
f.write("dtm_path=test_data_path+'dtm/'\n")
f.write("output_path=test_data_path+'output/'\n")
f.write("vtk_path=test_data_path+'vtk/'\n")
f.write("\n")
f.write("fault_file_csv=tmp_path+'fault_file.csv'\n")
f.write("structure_file_csv=tmp_path+'structure_file.csv'\n")
f.write("geology_file_csv=tmp_path+'geology_file.csv'\n")
f.write("mindep_file_csv=tmp_path+'mindep_file.csv'\n")
f.write("\n")
f.write("fault_file=data_path+fault_file\n")
f.write("fold_file=data_path+fold_file\n")
f.write("structure_file=data_path+structure_file\n")
f.write("geology_file=data_path+geology_file\n")
f.write("mindep_file=data_path+mindep_file\n")
f.write("\n")
f.write("strat_graph_file=test_data_path+'graph/graph_strat_NONE.gml'\n")
f.write("\n")
f.write("dtm_file=dtm_path+'dtm.tif'\n")
f.write("dtm_reproj_file=dtm_path+'dtm_rp.tif'\n")
f.write("\n")
f.write("if(not os.path.isdir(test_data_path)):\n")
f.write("   os.mkdir(test_data_path)\n")
f.write("if(not os.path.isdir(tmp_path)):\n")
f.write("   os.mkdir(tmp_path)\n")
f.write("if(not os.path.isdir(output_path)):\n")
f.write("   os.mkdir(output_path)\n")
f.write("if(not os.path.isdir(dtm_path)):\n")
f.write("   os.mkdir(dtm_path)\n")
f.write("if(not os.path.isdir(vtk_path)):\n")
f.write("   os.mkdir(vtk_path)\n")
f.write("if(not os.path.isdir(graph_path)):\n")
f.write("   os.mkdir(graph_path)\n")
f.write("\n")
f.write("print('Default parameters loaded from '+test_data_path+'m2l_config.py:')\n")
f.write("with open('../"+test_data_name+"/m2l_config.py', 'r') as myfile:\n")
f.write("  data = myfile.read()\n")
f.write("  print(data)\n")
f.write("  myfile.close()\n")
f.write("\n")
f.write("print('Modify these parameters in the cell below')\n")
f.close()